In [ ]:
"""
Purpose: To implement a proofreading rule that eliminates axon messes

Psuedocode: 
1) Get the concept graph of the axon 
2) Eliminate all of the nodes that are too long skeletal length
3) Divide the remaining axon into connected components
- if too many nodes are in the connected component then it is
an axon mess and should delete all those nodes




"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")

In [3]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

import numpy as np

import proofreading_utils as pru

import preprocessing_vp2 as pre

# -- For the axon classification --

import neuron_searching as ns

import skeleton_utils as sk

import numpy_utils as nu

import networkx_utils as xu

import system_utils as su

import classification_utils as clu
import proofreading_utils as pru

INFO - 2021-01-29 18:30:14,486 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-29 18:30:14,487 - settings - Setting database.user to celiib
INFO - 2021-01-29 18:30:14,488 - settings - Setting database.password to newceliipass
INFO - 2021-01-29 18:30:14,492 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-01-29 18:30:14,493 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-29 18:30:14,506 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-01-29 18:30:14,775 - settings - Setting enable_python_native_blobs to True


In [4]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-29 18:30:21,403 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-29 18:30:21,677 - settings - Setting enable_python_native_blobs to True


# Getting the Neuron

In [76]:
segment_id = 864691134884741370 #worked
#segment_id = 864691135578531524

In [77]:
neuron_obj = (minnie.Decomposition() & dict(segment_id=segment_id)).fetch1("decomposition")

Decompressing Neuron in minimal output mode...please wait


In [78]:
nviz.visualize_neuron_lite(neuron_obj)


 Working on visualization type: mesh


# Processing the Neuron

In [54]:
"""
1) Split the neuron
2) Look for really high degree splits
2) Identify the Axon


"""

'\n1) Split the neuron\n2) Look for really high degree splits\n2) Identify the Axon\n\n\n'

# Splittig the Neuron

In [79]:
import proofreading_utils as pru
split_neurons = pru.split_neuron(neuron_obj)

No error limbs to processs so just returning the original neuron


In [80]:
curr_neuron_obj = split_neurons[0]

In [81]:
curr_neuron_obj.n_error_limbs

0

# Developing the Axon Mess Function 

In [214]:
ret_group = nru.low_branch_length_clusters(curr_neuron_obj,
                         max_skeletal_length = 10000,
                        use_axon_like_restriction=False)
ret_group

{'L0': [73, 78, 81, 83, 84, 85], 'L4': [3, 8, 11, 17, 18]}

In [185]:
nviz.plot_limb_branch_dict(neuron_obj,
                          ret_group)


 Working on visualization type: mesh


In [219]:
pru.filter_away_low_branch_length_clusters(curr_neuron_obj,
                                           max_skeletal_length=10000,
                                           plot_limb_branch_filter_away=False,
                                           plot_limb_branch_filter_with_disconnect_effect=False,
                                          plot_final_neuron=True)



----Working on seperate_graph 0----


---------Working on soma_idx = 0, soma_group_idx 0, endpt = [ 571559.  687939. 1051520.]---------
Starting_edge inside branches_to_conept = [[ 571559.  687939. 1051520.]
 [ 572210.  686820. 1051460.]]
At the start, starting_node (in terms of the skeleton, that shouldn't match the starting edge) = [17]
printing out current edge:
[[ 571559.  687939. 1051520.]
 [ 572210.  686820. 1051460.]]
edge_endpoints_to_process was empty so exiting loop after 37 iterations
starting_node in concept map (that should match the starting edge) = 18
Total time for branches to concept conversion = 0.9459793567657471

Done generating concept network 




----Working on seperate_graph 0----


---------Working on soma_idx = 0, soma_group_idx 0, endpt = [ 569860.84988831  702506.41601337 1054316.10516109]---------
Starting_edge inside branches_to_conept = [[ 569860.84988831  702506.41601337 1054316.10516109]
 [ 583740.9851677   749746.91737799 1059022.87518891]]
At the st

(<neuron.Neuron at 0x7fd558559470>, 2651.9004269156217, 2175.1189226707065)

# Find all the high degree nodes

In [16]:
"""
Pseudocode: 
For each limb
1) Get the limb skeleton
2) Convert the limb skeleton to a graph
3) Find all nodes with a degree above five
4) Get the coordinates of all of those nodes
5) Graph the results

"""

'\nPseudocode: \nFor each limb\n1) Get the limb skeleton\n2) Convert the limb skeleton to a graph\n3) Find all nodes with a degree above five\n4) Get the coordinates of all of those nodes\n5) Graph the results\n\n'

In [18]:
import skeleton_utils as sk
import networkx_utils as xu

In [21]:
curr_high_degree_nodes

[]

In [50]:
limb_high_degree_coordinates = []
max_degree = 4
verbose = True

for i,limb in enumerate(curr_neuron_obj):
    
    if verbose:
        print(f"--- Working on Limb {i} ---")
    
    #1) Get the limb skeleton
    #2) Convert the limb skeleton to a graph
    limb_sk_gr = sk.convert_skeleton_to_graph(limb.skeleton)
    
    #3) Find all nodes with a degree above five
    curr_high_degree_nodes = xu.get_nodes_greater_or_equal_degree_k(limb_sk_gr,max_degree)
    
    if verbose:
        print(f"curr_high_degree_nodes = {curr_high_degree_nodes}")
    
    if len(curr_high_degree_nodes)>0:
    
        #4) Get the coordinates of all of those nodes
        curr_high_degree_coordinates = xu.get_coordinate_by_graph_node(limb_sk_gr,curr_high_degree_nodes)

        if verbose:
            print(f"curr_high_degree_coordinates = {curr_high_degree_coordinates}")

        limb_high_degree_coordinates += list(curr_high_degree_coordinates)
        

[autoreload of proofreading_utils failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/usr/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 781, in get_code
  File "<frozen importlib._bootstrap_external>", line 741, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/meshAfterParty/proofreading_utils.py", line 2647
    plot_final_neuron=Fals

--- Working on Limb 0 ---
curr_high_degree_nodes = []
--- Working on Limb 1 ---
curr_high_degree_nodes = []
--- Working on Limb 2 ---
curr_high_degree_nodes = []
--- Working on Limb 3 ---
curr_high_degree_nodes = []
--- Working on Limb 4 ---
curr_high_degree_nodes = []
--- Working on Limb 5 ---
curr_high_degree_nodes = []
--- Working on Limb 6 ---
curr_high_degree_nodes = []
--- Working on Limb 7 ---
curr_high_degree_nodes = []
--- Working on Limb 8 ---
curr_high_degree_nodes = []


In [24]:
#5) Graph the results
nviz.visualize_neuron_lite(curr_neuron_obj,
                           scatters=limb_high_degree_coordinates,
                     scatter_size=1)


 Working on visualization type: mesh


In [42]:
nviz.visualize_neuron(neuron_minus_mess[0],
                     limb_branch_dict=dict(L0="all"),
                     )


 Working on visualization type: mesh

 Working on visualization type: skeleton
